# Define the methods

In [43]:
import requests
import json
import time
from functools import wraps

def timer(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function '{func.__name__}' executed in {execution_time:.4f} seconds")
        return result
    return wrapper


BASE_URL = 'http://localhost:8001/api'


def start_conversation(connection_id):
    response = requests.post(f"{BASE_URL}/start_conversation", params={"connection_id": connection_id})
    if response.status_code == 200:
        return response.json()['id']
    else:
        raise Exception("Failed to start conversation")


@timer
def send_message(conversation_id, input_text):
    print('processing.....\n')
    response = requests.post(f"{BASE_URL}/send_message", params={"input": input_text, "conversation_id": conversation_id})
    if response.status_code == 200:
        # print(response.json())
        return response.text
    else:
        raise Exception("Failed to send message")


def display_messages(messages:list):
    for message in messages:
        content = message[1]
        # print(f'############/n{content}')
        if message[0] == 'ai':
            try:
                outputs = json.loads(content)
                sql, knowledge, thought = outputs['sql'], outputs['knowledge'], outputs['thought']
                print(f'AI: \n############ SQL ############ \n{sql} \n############ knowledge ############ \n{knowledge} \n############ thoughts ############ \n{thought} \n\n\n')
            except Exception as e:
                # print(e)
                print(f'AI: \n{content}\n\n\n')
        if message[0] == 'human':
            print(f'YOU: \n{content}\n\n\n')


def get_history(conversation_id):
    response = requests.get(f"{BASE_URL}/conversation_history", params={"conversation_id": conversation_id})
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to retrieve conversation history")
    

def verify(conversation_id):
    response = requests.post(f"{BASE_URL}/verify", params={"conversation_id": conversation_id})
    if response.status_code == 200:
        # return response.json()
        return "knowledgebase updated"
    else:
        raise Exception("Failed to add knowledge")


# def get_history(conversation_id):
    
# conversation_id=None

def chat(connection_id):
    global conversation_id
    
    if not conversation_id == None:
        history = get_history(conversation_id)
        display_messages(history)
    else:
        # global conv_id
        conversation_id = start_conversation(connection_id)
        # conv_id = conversation_id

    while True:
        user_input = input("You: ")
        display_messages([('human', user_input)])
        if user_input.lower() == 'exit'or user_input== '':
            break
        response = send_message(conversation_id, user_input)
        print('done!\n')
        display_messages([('ai', response)])
    





# Start a new chat

In [ ]:
conversation_id = None

chat(connection_id='1')

YOU: 
which artist has the highest APOT



processing.....

done!

AI: 
"It seems that there is no direct column named \"APOT\" in the available tables. However, \"APOT\" might refer to a calculated value based on sales or popularity of tracks associated with artists.\n\nTo find the artist with the highest sales (which could be interpreted as APOT), I can calculate the total sales for each artist based on the `InvoiceLine` and `Track` tables, and then join that with the `Artist` table.\n\nWould you like me to proceed with this approach?"



YOU: 






In [ ]:
# global variable conversation_id will change after calling chat
this_id = conversation_id
conversation_id

11

# Resume existing chat

In [38]:
# Just call chat() directly to resume the existing chat
chat(connection_id='1')

YOU: 
which artist has the highest APOT



AI: 
It seems that there is no direct column named "APOT" in the available tables. However, "APOT" might refer to a calculated value based on sales or popularity of tracks associated with artists.

To find the artist with the highest sales or popularity, we would typically need to aggregate sales data from the `InvoiceLine` table, which contains information about the quantity and price of tracks sold, and join it with the `Track` and `Artist` tables to get the artist information.

Would you like me to proceed with this approach to find the artist with the highest sales?



YOU: 
APOT stands for average price of tracks



AI: 
############ SQL ############ 
SELECT a.ArtistId, a.Name AS ArtistName, AVG(t.UnitPrice) AS AveragePrice
FROM Artist a
JOIN Album al ON a.ArtistId = al.ArtistId
JOIN Track t ON al.AlbumId = t.AlbumId
GROUP BY a.ArtistId
ORDER BY AveragePrice DESC
LIMIT 1; 
############ knowledge ############ 
#knowledge about database
* T

In [39]:
conversation_id

11

# verification

Adding long term memory to the agent

* ### if no long term memory. start a new chat, the agent forgets what it learned from past conversation

In [40]:
conversation_id = None

chat(connection_id='1')

YOU: 
which artist has the highest APOT



processing.....

done!

AI: 
"It seems that there is no direct column named \"APOT\" in the available tables. However, \"APOT\" might refer to a calculated value based on sales or popularity of tracks associated with artists.\n\nTo find the artist with the highest sales (which could be interpreted as APOT), I can calculate the total sales for each artist based on the `InvoiceLine` and `Track` tables, and then join that with the `Artist` table.\n\nWould you like me to proceed with this approach?"



YOU: 






* ### verify() function add the knowledge learned from conversation to the knowledgebase to form long term memory

In [49]:
conversation_id = this_id
verify(conversation_id=conversation_id)

'knowledgebase updated'

* ### Then ask similar questions in new conversations. The agent can understand what APOT is and where to find date info at the first round of conversation without needing user to explain.

In [50]:
conversation_id = None

chat(connection_id='1')



YOU: 
which artist has the highest APOT in year 2021



processing.....

Function 'send_message' executed in 71.3734 seconds
done!

AI: 
############ SQL ############ 
SELECT a.ArtistId, a.Name AS ArtistName, AVG(t.UnitPrice) AS AveragePrice
FROM Artist a
JOIN Album al ON a.ArtistId = al.ArtistId
JOIN Track t ON al.AlbumId = t.AlbumId
JOIN InvoiceLine il ON t.TrackId = il.TrackId
JOIN Invoice i ON il.InvoiceId = i.InvoiceId
WHERE strftime('%Y', i.InvoiceDate) = '2021'
GROUP BY a.ArtistId
ORDER BY AveragePrice DESC
LIMIT 1; 
############ knowledge ############ 
#knowledge about database
* The Artist table contains information about artists, including ArtistId and Name.
* The Album table contains information about albums, including AlbumId and ArtistId.
* The Track table contains information about tracks, including TrackId, AlbumId, and UnitPrice.
* The Invoice table contains information about invoices, including InvoiceId and InvoiceDate.
* The InvoiceLine table contains information abo